In [1]:
import numpy as np
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution2D, MaxPooling2D, Flatten, Dropout
from keras import optimizers

Using TensorFlow backend.


In [2]:
batch_size = 64
num_classes = 10
epochs = 1

In [3]:
# input image dimensions
img_rows, img_cols = 28, 28

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(-1,1,28,28)/255
x_test = x_test.reshape(-1,1,28,28)/255

In [5]:
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 1, 28, 28)
60000 train samples
10000 test samples


In [6]:
# convert class vectors to binary class matrices
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

In [7]:
# model construction
model = Sequential()
model.add(Convolution2D(
    input_shape=( 1, 28, 28), # height width
    filters=32,
    kernel_size=5,
    strides=1,
    padding='same',      # Padding method
    data_format='channels_first',
))
model.add(Activation('relu'))
model.add(MaxPooling2D(
	pool_size=2,
	strides=2,

))
model.add(Convolution2D(
	64,5,strides=1,padding='same',data_format='channels_first'
))
model.add(Activation('relu'))
model.add(MaxPooling2D(
	2,2,'same'
))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [8]:
adam = optimizers.Adam(lr=1e-4)
model.compile(adam,'categorical_crossentropy',metrics=['accuracy'])
model.fit(x_train[:1000],y_train[:1000],batch_size=batch_size,epochs=epochs)

Epoch 1/1
1000/1000 [==============================] - 9s 9ms/step - loss: 2.1670 - acc: 0.2790


In [9]:
loss, accuracy = model.evaluate(x_test[:100],y_test[:100])
print('test loss :', loss)
print('test acc:', accuracy)

100/100 [==============================] - 0s 4ms/step
test loss : 1.917460699081421
test acc: 0.62
